In [14]:
import ipywidgets as w

In [15]:
intro_box = w.VBox(children=(w.HTML(value='<font size="+2"><b>CARA</b> Covid Calculator</font>'),
                         w.Text(placeholder="E.g. Workshop without masks", description="Simulation name:",
                                style={'description_width': '35%'}),
                         w.Text(placeholder="E.g. 17/R-033", description="Room number:",
                                style={'description_width': '35%'})))

In [16]:
volume_text = w.Text(placeholder="Room volume (m³)", description="Room volume")
area_text = w.Text(placeholder="Room floor area (m²)", description="Floor area")
height_text = w.Text(placeholder="Room ceiling height (m²)", description="Ceiling height")

room_dimensions_box = w.VBox(children=(volume_text, w.Label(value="------- OR -------"),area_text, height_text),
                             layout=w.Layout(align_items='center'))
def manage_room_text_fields(_):
    volume_text.disabled = bool(area_text.value) or bool(height_text.value)
    area_text.disabled = bool(volume_text.value)
    height_text.disabled = bool(volume_text.value)

for text in [volume_text, area_text, height_text]:
    text.observe(manage_room_text_fields)
    text.style.description_width = "50%"

volume_text.observe(manage_room_text_fields)
area_text.observe(manage_room_text_fields)
height_text.observe(manage_room_text_fields)

In [22]:
mechanical_button = w.ToggleButton(value=False, description='Mechanical ventilation', layout={'width': '100%'})
acph_text = w.Text(description="Air changes per hour", style={'description_width': 'aut'})
asfr_text = w.Text(description="Air supply flow rate", style={'description_width': '45%'})

def toggle_mechanical(b: w.Button):
    acph_text.disabled = not (mechanical_button.value or asfr_text.value)
    asfr_text.disabled = not (mechanical_button.value or acph_text.value)

w.jsdlink((acph_text, 'value'), (asfr_text, 'disabled'))
w.jsdlink((asfr_text, 'value'), (acph_text, 'disabled'))
acph_text.disabled = True
asfr_text.disabled = True

mechanical_button.observe(toggle_mechanical)

mech_vent_box = w.VBox(
    children=(mechanical_button, acph_text, w.Label(value="------- OR -------"), asfr_text),
    layout=w.Layout(align_items='center')
)